#### Setup Param

In [ ]:
//  pass the parameters
dbutils.widgets.text("source_catalog", "ag_content_ims_acs")
dbutils.widgets.text("source_environment", "prod")
dbutils.widgets.text("source_version", "")

dbutils.widgets.text("target_catalog", "ag_ra_search_analytics_data")
dbutils.widgets.text("target_environment", "dev")
dbutils.widgets.text("target_version", "v1_0")
dbutils.widgets.text("pipeline_name", "")


dbutils.widgets.text("metadata_table_prefix", "")

// dynamic paramters
val source_catalog = dbutils.widgets.get("source_catalog")
val source_environment = dbutils.widgets.get("source_environment")
val source_version = dbutils.widgets.get("source_version")
val target_catalog = dbutils.widgets.get("target_catalog")
val target_environment = dbutils.widgets.get("target_environment")
val target_version = dbutils.widgets.get("target_version")
val pipeline_name = dbutils.widgets.get("pipeline_name")
val metadata_table_prefix = dbutils.widgets.get("metadata_table_prefix")



#### SchemaResolver

In [ ]:

object SchemaResolver {

  private def getWidget(
    name: String, 
    default: String
  ): String = {

    try {
      val value = dbutils.widgets.get(name)
      if (value == null || value.isEmpty) default else value
    } catch {
      case _: Throwable => default
    }
  }

  private val source_catalog = getWidget("source_catalog", "ag_content_ims_acs")
  private val source_environment = getWidget("source_environment", "prod")
  private val source_version = getWidget("source_version", "")

  private val target_catalog = getWidget("target_catalog", "ag_ra_search_analytics_data")
  private val target_environment = getWidget("target_environment", "dev")
  private val target_version = getWidget("target_version", "v1_0")
  private val metadata_table_prefix = getWidget("metadata_table_prefix", "v1_0")

  private val dapSchemaBases = Seq(

      "dap_entity_wos",
      "dap_metrics_wos",
      "dap_entity_pprn",
      "dap_metrics_pprn",
      "dap_docs",
      "dap_reference",
      "dap_sort_ref",
      "dap_entity_enrich",
      "dap_grant",
      "dap_prod_core",
      "dap_ops",
      "dap_work"
    )

  private val acsSchemaBases= Seq(
    
      "gold_entity",
      "gold_wos",
      "gold_pprn"
  )

  // define constant values initialized only once atstartup
  val PIPELINE = getWidget("pipeline_name", "")
  val OPS_SCHEMA = s"${target_catalog}_${target_version}.dap_ops_${target_version}"
  val OPS_TABLE_PREFIX: String = Option(metadata_table_prefix)
    .filter(_.trim.nonEmpty)
    .map(p => s"${p}_")
    .getOrElse("")


  val ACS_SCHEMA_MAP: Map[String, String]  = {

     val version = if(source_version.isEmpty) "" else  s"_${source_version}"
     Map(
      "entity" -> s"${source_catalog}_${source_environment}.gold_entity${version}",
      "wos" -> s"${source_catalog}_${source_environment}.gold_wos${version}",
      "pprn" -> s"${source_catalog}_${source_environment}.gold_pprn${version}",
      "dap" -> s"${target_catalog}_${target_environment}.sandbox_${target_version}",
    )
  }

  val DAP_SCHEMAS: Seq[String] = {

    val base = s"${target_catalog}_${target_environment}"
    dapSchemaBases.map { key =>
      s"$base.${key}_$target_version"
    }
  }

  val ACS_SCHEMAS: Seq[String] = {

    val version = if(source_version.isEmpty) "" else  s"_${source_version}"
    val base = s"${source_catalog}_${source_environment}"

    acsSchemaBases.map { key =>
      s"$base.${key}${version}"
    }
  }
}



#### Test

In [ ]:


SchemaResolver.ACS_SCHEMA_MAP.foreach{ case (k,v) => println(s"$k -> $v") }
println("------")
SchemaResolver.DAP_SCHEMAS.foreach{ schema => println(schema) }  
println("------")
SchemaResolver.ACS_SCHEMAS.foreach{ schema => println(schema) }  
println("------")

println(SchemaResolver.PIPELINE)